# ReGenNexus Core - Basic Demo

This notebook demonstrates the basic functionality of ReGenNexus Core.
It shows how to establish connections, send messages, and handle events.

In [ ]:
import asyncio
import json
import time
import uuid
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
import nest_asyncio

# Apply nest_asyncio to allow asyncio to work in Jupyter
nest_asyncio.apply()

In [ ]:
# Import ReGenNexus Core modules
from regennexus.protocol.client import Client
from regennexus.protocol.message import Message
from regennexus.protocol.protocol_core import MessageType

In [ ]:
# Create output area for logs
log_output = widgets.Output()
display(HTML("<h2>ReGenNexus Core Basic Demo</h2>"))
display(HTML("<h3>Log Output:</h3>"))
display(log_output)

In [ ]:
# Initialize clients
client1 = None
client2 = None

async def setup_demo():
    """Set up the demo environment with two clients."""
    global client1, client2
    
    with log_output:
        print("Setting up demo environment...")
        
        # Create clients
        client1 = Client(client_id="demo_client_1")
        client2 = Client(client_id="demo_client_2")
        
        # Set up message handlers
        client1.set_message_handler(handle_client1_message)
        client2.set_message_handler(handle_client2_message)
        
        # Initialize clients
        await client1.initialize()
        await client2.initialize()
        
        print("Clients initialized")
        print("Demo environment ready!")

async def handle_client1_message(message):
    """Handle messages received by client1."""
    with log_output:
        print(f"Client 1 received: {message.content} (type: {message.message_type})")

async def handle_client2_message(message):
    """Handle messages received by client2."""
    with log_output:
        print(f"Client 2 received: {message.content} (type: {message.message_type})")

async def send_message(from_client, to_client, content, message_type=MessageType.TEXT):
    """Send a message from one client to another."""
    with log_output:
        print(f"Sending message from {from_client.client_id} to {to_client.client_id}")
        print(f"Content: {content}")
        
        # Create message
        message = Message(
            sender_id=from_client.client_id,
            recipient_id=to_client.client_id,
            message_type=message_type,
            content=content
        )
        
        # Send message
        await from_client.send_message(message)
        
        print("Message sent!")

async def cleanup_demo():
    """Clean up the demo environment."""
    global client1, client2
    
    with log_output:
        print("Cleaning up demo environment...")
        
        # Shut down clients
        if client1:
            await client1.shutdown()
        
        if client2:
            await client2.shutdown()
            
        print("Clients shut down")
        print("Demo environment cleaned up!")

In [ ]:
# Run setup
asyncio.run(setup_demo())

In [ ]:
# Create UI controls
message_input = widgets.Text(description="Message:", placeholder="Enter message here", layout=widgets.Layout(width="50%"))
client1_to_client2_button = widgets.Button(description="Client 1 → Client 2")
client2_to_client1_button = widgets.Button(description="Client 2 → Client 1")
clear_button = widgets.Button(description="Clear Log")
cleanup_button = widgets.Button(description="Cleanup Demo")

# Define button callbacks
def on_client1_to_client2_button_clicked(b):
    asyncio.ensure_future(send_message(client1, client2, message_input.value))

def on_client2_to_client1_button_clicked(b):
    asyncio.ensure_future(send_message(client2, client1, message_input.value))

def on_clear_button_clicked(b):
    log_output.clear_output()

def on_cleanup_button_clicked(b):
    asyncio.ensure_future(cleanup_demo())

# Connect callbacks
client1_to_client2_button.on_click(on_client1_to_client2_button_clicked)
client2_to_client1_button.on_click(on_client2_to_client1_button_clicked)
clear_button.on_click(on_clear_button_clicked)
cleanup_button.on_click(on_cleanup_button_clicked)

# Display UI
display(message_input)
display(widgets.HBox([
    client1_to_client2_button, client2_to_client1_button, clear_button, cleanup_button
]))

## Instructions

1. Enter a message in the text field
2. Click "Client 1 → Client 2" to send a message from Client 1 to Client 2
3. Click "Client 2 → Client 1" to send a message from Client 2 to Client 1
4. Watch the log output to see messages being sent and received
5. Click "Clear Log" to clear the log output
6. Click "Cleanup Demo" when finished to release resources

This demo shows the basic communication capabilities of ReGenNexus Core.

In [ ]:
# Advanced: Send a JSON message
async def send_json_message():
    json_content = {
        "command": "ping",
        "timestamp": time.time(),
        "data": {
            "value": 42,
            "unit": "answer"
        }
    }
    
    await send_message(
        from_client=client1,
        to_client=client2,
        content=json.dumps(json_content),
        message_type=MessageType.JSON
    )

# Create and display button
json_button = widgets.Button(description="Send JSON Message")
json_button.on_click(lambda b: asyncio.ensure_future(send_json_message()))
display(json_button)